In [47]:
import numpy as np
import  pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import mnist
from keras.models import Sequential
from keras.layers import Dense,Dropout, Activation,Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator


In [48]:
import os
import numpy as np
from PIL import Image
import pickle
import cv2
import matplotlib.pyplot as plt

BASE_DIR= os.path.dirname(os.path.abspath("file")) 
image_dir=os.path.join(BASE_DIR,"image") 
recognizer= cv2.face.LBPHFaceRecognizer_create()
current_id=0
label_id={}
y_labels=[]
x_train=[]
for root,dirs,files in os.walk(image_dir):
    for file in files:
        if file.endswith("png"):
            path=os.path.join(root,file)
            label=os.path.basename(os.path.dirname(path)).replace(" ","_").lower()
        
#             print(label,path)
            
            if label in label_id:
                pass
            else:
                label_id[label]=current_id
                current_id+=1
            id_=label_id[label]  
#             print(label_id)
#           y_labels.append(label)
#           x_train.append(path)
            pil_image=Image.open(path).convert("L")#gray
            image=cv2.resize(np.uint8(pil_image),(45,45),interpolation=cv2.INTER_AREA)
            
#             plt.imshow(image)
#             plt.show()
            image_array=image.reshape(45,45,1)
#             image_array=np.reshape(image_array,(4096,))
#             print(image_array)
            x_train.append(image_array)
            y_labels.append(id_)
#             plt.imshow(image_array[:,:,0])
#             plt.show()
            
#print(y_labels)
# x_train=np.array(x_train)
x_train=np.array(x_train)
y_labels_orig=np.array(y_labels).reshape(len(y_labels))
print(x_train.shape)
print(y_labels_orig)

with open("labels.pickle","wb")as f:
    pickle.dump(label_id,f)

(251, 45, 45, 1)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]


In [49]:
number_of_classes = 10
Y_train = np_utils.to_categorical(y_labels_orig,number_of_classes)

In [50]:
X_train = x_train.astype('float32')


X_train/=255

In [51]:
X_train.shape

(251, 45, 45, 1)

In [52]:
Y_train.shape

(251, 10)

In [2]:
# X1,y1,X2,y2=mnist.load()

In [3]:
# X1=X1.reshape(60000,28,28)
# X2=X2.reshape(10000,28,28)
# y_test=y2.reshape(-1,1)
# y_train=y1.reshape(-1,1)

# X_train=np.zeros((60000,45,45))
# for i in range(np.shape(X1)[0]):
#     X_train[i]=cv2.resize(X1[i], (45,45), interpolation = cv2.INTER_CUBIC)
    
# X_test=np.zeros((10000,45,45))
# for i in range(np.shape(X2)[0]):
#     X_test[i]=cv2.resize(X2[i], (45,45), interpolation = cv2.INTER_CUBIC)
# X_train=X_train.reshape(-1,45,45,1)  
 
# X_test=X_test.reshape(-1,45,45,1)  
# X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')

# X_train/=255
# X_test/=255
# number_of_classes = 10
# Y_train = np_utils.to_categorical(y_train,number_of_classes)
# Y_test = np_utils.to_categorical(y_test,number_of_classes)

In [53]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(45,45,1)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

BatchNormalization(axis=-1)
model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
# Fully connected layer

BatchNormalization()
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.2))
model.add(Dense(10))

# model.add(Convolution2D(10,3,3, border_mode='same'))
# model.add(GlobalAveragePooling2D())
model.add(Activation('softmax'))


In [54]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])


In [55]:
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)

test_gen = ImageDataGenerator()


In [61]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(X_train,Y_train,test_size=0.45)

In [62]:
train_generator = gen.flow(train_x, train_y, batch_size=10)
test_generator = test_gen.flow(test_x, test_y, batch_size=10)


In [63]:
hist=model.fit_generator(train_generator, steps_per_epoch=154//10, epochs=100, 
                    validation_data=test_generator, validation_steps=113//10)


Epoch 1/100
15/15 [==============================] - 1s 65ms/step - loss: 0.4640 - accuracy: 0.9054 - val_loss: 0.1215 - val_accuracy: 0.9545
Epoch 2/100
15/15 [==============================] - 1s 72ms/step - loss: 0.3964 - accuracy: 0.8649 - val_loss: 0.2902 - val_accuracy: 0.9000
Epoch 3/100
15/15 [==============================] - 1s 66ms/step - loss: 0.3740 - accuracy: 0.8851 - val_loss: 0.1371 - val_accuracy: 0.9545
Epoch 4/100
15/15 [==============================] - 1s 68ms/step - loss: 0.1257 - accuracy: 0.9730 - val_loss: 0.1576 - val_accuracy: 0.9636
Epoch 5/100
15/15 [==============================] - 1s 66ms/step - loss: 0.2289 - accuracy: 0.9324 - val_loss: 0.1727 - val_accuracy: 0.9545
Epoch 6/100
15/15 [==============================] - 1s 67ms/step - loss: 0.0909 - accuracy: 0.9662 - val_loss: 0.1573 - val_accuracy: 0.9636
Epoch 7/100
15/15 [==============================] - 1s 72ms/step - loss: 0.1149 - accuracy: 0.9595 - val_loss: 0.1345 - val_accuracy: 0.9727
Epoch 

In [64]:
score = model.evaluate(test_x, test_y)
print()
print('Test accuracy: ', score[1])

4/4 [==============================] - 0s 19ms/step - loss: 0.2714 - accuracy: 0.9469

Test accuracy:  0.9469026327133179


In [12]:
predictions = model.predict_classes(X_test)

predictions = list(predictions)
actuals = list(Y_test)

sub = pd.DataFrame({'Actual': actuals, 'Predictions': predictions})
sub.to_csv('./output_cnn.csv', index=False)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [65]:
model.save("Newmodel2_train.h5",hist)